# Imports

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import time
import re
import csv
import pymorphy2
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import os

### Спарсим рамблер

In [20]:
signs = ['aries', 'gemini', 'taurus', 'cancer', 'leo', 'virgo', 'libra', 'scorpio', 'sagittarius', 'capricorn',
         'aquarius', 'pisces']
data = pd.DataFrame(columns=['date', 'sign', 'text'])
# data = pd.read_csv(names=['date', 'sign', 'text'])

for i in range(2004, 2021):
    for j in range(1, 13):
        for k in range(1, 32):
            for sign in signs:
                try:
                    year = i
                    month = j
                    day = k
                    if month < 10:
                        month = '0' + str(month)
                    if day < 10:
                        day = '0' + str(day)
                    date = '%s-%s-%s' % (year, month, day)
                    url = 'https://horoscopes.rambler.ru/%s/%s/?updated' % (sign, date)  # url для второй страницы
                    print(url)
                    r = requests.get(url)
                    response = r.text.encode('utf-8')

                    soup = BeautifulSoup(response, features="lxml")
                    text = soup.find('div', {'class': '_1dQ3'}).text
                    data = data.append({'date': date, 'sign': sign, 'text': text}, ignore_index=True)
                    data.to_csv('data.csv', encoding='utf-8')
                    print('saved date=', date, ' sign=', sign)
                except Exception:
                    continue

https://horoscopes.rambler.ru/aries/2004-01-01/?updated
https://horoscopes.rambler.ru/gemini/2004-01-01/?updated
https://horoscopes.rambler.ru/taurus/2004-01-01/?updated
https://horoscopes.rambler.ru/cancer/2004-01-01/?updated
https://horoscopes.rambler.ru/leo/2004-01-01/?updated
https://horoscopes.rambler.ru/virgo/2004-01-01/?updated
https://horoscopes.rambler.ru/libra/2004-01-01/?updated
https://horoscopes.rambler.ru/scorpio/2004-01-01/?updated
https://horoscopes.rambler.ru/sagittarius/2004-01-01/?updated


KeyboardInterrupt: 

### Спарсим пару пабликов в ВК

In [18]:
import pprint
# id групп, которые я планирую парсить
groups = {
    'goroscop_love_id': '-39461653',
    'unformat' : '-27726363',
    'public65722210': '-65722210'
}
# ссылка на документацию в README
vk_token = "yourtoken"

zodiac_signs = ['Рыбы', 'Водолей', 'Козерог', 'Стрелец', 'Скорпион', 'Весы',
                'Дева', 'Лев', 'Рак', 'Близнецы', 'Телец', 'Овен']

separator = {
    '-39461653': ['❤', 0, -2, '\n \n', -1],
    '-27726363': ['|',0 , -1, '\n \n', -1],
    '-65722210': ['❤', 0, -2, '\n \n', -1]
}

def vk_parser(id, token, offset):
    url = f"https://api.vk.com/method/wall.get?v=5.131&count=100&owner_id=" \
          f"{id}&offset={offset}&access_token={token}"
    result = requests.get(url).json()
    return result


def get_data(result, group_id):
    dir_exists = os.path.exists('data')
    if not dir_exists:
        os.mkdir('data')
    if 'response' in result:
        for wall_text in result['response']['items']:
            if wall_text['text'] != "":
                for sign in zodiac_signs:
                    # if sign in wall_text['text']:

                    if sign == wall_text['text'].split(separator[group_id][0])[0]\
                        [0:separator[group_id][2]] and \
                            len(wall_text['text'].split("\n")[0]) < 25:
                        if f"{separator[group_id][3]}" in wall_text['text']:
                            lol = [wall_text['text'].split()[0],
                                   wall_text['text'].split(f"{separator[group_id][3]}") \
                                       [separator[group_id][4]].replace("\n","")]
                        else:
                            lol = [wall_text['text'].split()[0],
                                   wall_text['text'].split(f"\n") \
                                       [separator[group_id][4]].replace("\n","")]
                        if not os.path.isfile(f'{path}/data/all_signs.csv'):
                            with open(f'{path}/data/all_signs.csv', 'w', newline="", encoding="utf-8") as File:
                                writer = csv.writer(File, delimiter=';')
                                writer.writerow(['sign', "text"])
                        with open(f'{path}/data/all_signs.csv', 'a', encoding="utf-8", newline='') as file:
                            writer = csv.writer(file, delimiter=';')
                            writer.writerow(lol)

for group_id in groups.values():
    print("sleep")
    time.sleep(3)                      ###Чтобы не возникала ошибка 'Too many requests per second'
    res = vk_parser(group_id, vk_token, 0)
    print(res['response']['count'])
    print(res['response']['count'] // 100)
    for i in range(res['response']['count'] // 100+1):
        piece_of_data = vk_parser(group_id, vk_token, i * 100)
        get_data(piece_of_data, group_id)

sleep
55628
556
sleep
49426
494
sleep
18934
189


### Приводим данные к одному виду и объединяем в один датасет

In [22]:
Horoscope = pd.read_csv(f"{path}/data/all_signs.csv", sep=';')
Horoscope.shape

(62517, 2)

In [29]:
data = pd.read_csv(f'{path}/data/data.csv')
data = data.drop("date", axis = 1)
data.shape

(57243, 2)

In [34]:
data.head(1)

,sign,text
0,aries,"Любые разногласия во мнениях скоро улягутся, а..."


In [55]:
Horoscope.head(1)

,sign,text
0,Рыбы.,Завтра в делах любовных Рыбам необходимо вспом...


In [56]:
Horoscope.sign.value_counts()

Телец.       1914
Козерог.     1912
Лев.         1910
Близнецы.    1910
Овен.        1910
Дева.        1909
Скорпион.    1909
Водолей.     1909
Стрелец.     1909
Рыбы.        1907
Рак.         1906
Весы.        1905
Овен         1234
Весы         1232
Скорпион     1232
Водолей      1231
Рак          1231
Дева         1231
Близнецы     1231
Телец        1230
Рыбы         1230
Козерог      1229
Стрелец      1229
Лев          1228
Name: sign, dtype: int64

In [4]:
Horoscope['sign'] = Horoscope.sign.apply(lambda name: re.sub(r'[^\w\s]','', name))

In [5]:
Horoscope.sign.value_counts()

Овен        3144
Телец       3144
Козерог     3141
Близнецы    3141
Скорпион    3141
Водолей     3140
Дева        3140
Стрелец     3138
Лев         3138
Весы        3137
Рыбы        3137
Рак         3137
Name: sign, dtype: int64

In [7]:
Horoscope.loc[ Horoscope['sign'] == 'Рыбы', 'sign'] = 'Pisces'
Horoscope.loc[ Horoscope['sign'] == 'Дева', 'sign'] = 'Virgo'
Horoscope.loc[ Horoscope['sign'] == 'Весы', 'sign'] = 'Libra'
Horoscope.loc[ Horoscope['sign'] == 'Близнецы', 'sign'] = 'Gemini'
Horoscope.loc[ Horoscope['sign'] == 'Овен', 'sign'] = 'Aries'
Horoscope.loc[ Horoscope['sign'] == 'Рак', 'sign'] = 'Cancer'
Horoscope.loc[ Horoscope['sign'] == 'Телец', 'sign'] = 'Taurus'
Horoscope.loc[ Horoscope['sign'] == 'Козерог', 'sign'] = 'Capricorn'
Horoscope.loc[ Horoscope['sign'] == 'Скорпион', 'sign'] = 'Scorpio'
Horoscope.loc[ Horoscope['sign'] == 'Стрелец', 'sign'] = 'Sagittarius'
Horoscope.loc[ Horoscope['sign'] == 'Водолей', 'sign'] = 'Aquarius'
Horoscope.loc[ Horoscope['sign'] == 'Лев', 'sign'] = 'Leo'

In [8]:
Horoscope.sign.value_counts()

Aries          3144
Taurus         3144
Scorpio        3141
Capricorn      3141
Gemini         3141
Aquarius       3140
Virgo          3140
Sagittarius    3138
Leo            3138
Cancer         3137
Libra          3137
Pisces         3137
Name: sign, dtype: int64

In [71]:
data['sign'] = data.sign.apply(lambda name:  name.capitalize())

In [72]:
data.head()

,sign,text
0,Aries,"Любые разногласия во мнениях скоро улягутся, а..."
1,Gemini,Первый день нового года прекрасно подходит для...
2,Taurus,Этот день как нельзя лучше подходит для улучше...
3,Cancer,"Первый день нового года, да еще понедельник бу..."
4,Leo,Львы не сорвутся в начале года с места в карье...


In [73]:
result_df = pd.concat([data, Horoscope])
result_df.reset_index(inplace=True, drop=True)
result_df.shape

(111155, 2)

In [74]:
result_df

,sign,text
0,Aries,"Любые разногласия во мнениях скоро улягутся, а..."
1,Gemini,Первый день нового года прекрасно подходит для...
2,Taurus,Этот день как нельзя лучше подходит для улучше...
3,Cancer,"Первый день нового года, да еще понедельник бу..."
4,Leo,Львы не сорвутся в начале года с места в карье...
...,...,...
111150,Scorpio,Завтра состояние души Скорпиона будет напрямую...
111151,Sagittarius,Завтра есть смысл насильно заставить себя прео...
111152,Capricorn,Завтра Козерогу даже на романтическом свидании...
111153,Aquarius,Завтра звезды советуют Водолею не прятать голо...


In [76]:
result_df.sign.value_counts()

Taurus         9269
Aries          9268
Gemini         9266
Virgo          9264
Capricorn      9263
Scorpio        9262
Leo            9262
Cancer         9262
Aquarius       9261
Sagittarius    9261
Pisces         9259
Libra          9258
Name: sign, dtype: int64

In [84]:
result_df.to_csv(f'{path}/Horoscope_from_parser.csv', sep=',')

In [82]:
Horoscope = pd.read_csv(f'{path}/Horoscope_from_parser.csv', sep=',', index_col=0)

### Начинаем очистку текста


Первым делом убираем все повторения, так же некоторые строчки начинаются с маленькой буквы, это говорит о том, что гороскоп был разделен на несколько строчек, их мы так же уберем. После этого стоило бы убрать слишком короткие гороскопы.

In [83]:
Horoscope=Horoscope.drop_duplicates()
Horoscope.shape

(91633, 2)

In [84]:
Horoscope['lower'] = Horoscope.text.apply(lambda name:  name.lower() == name )
Horoscope.lower.value_counts()

False    91541
True        92
Name: lower, dtype: int64

In [85]:
Horoscope[Horoscope['lower'] == True]

,sign,text,lower
94631,Aquarius,"горячо и отличаются верностью к тем, кого выбр...",True
94632,Capricorn,исполняться как по волшебству!,True
94641,Aries,произойдет самая красивая сказка о любви.,True
100371,Aquarius,они ждут сумасшедших поступков..,True
100387,Gemini,кто хочет проводить его с вами.,True
...,...,...,...
107766,Scorpio,"вам сейчас кажется, что вы уже никогда не буде...",True
107769,Cancer,"солнечный удар, а кому-то вообще не светит..",True
107880,Capricorn,"это ваша очень большая, чертовски большая ошибка.",True
107899,Taurus,уже совсем скоро вам предстоит вместе засыпать.,True


In [86]:
Horoscope = Horoscope.loc[Horoscope['lower'] == False].drop("lower", axis = 1)
Horoscope

,sign,text
0,Aries,"Любые разногласия во мнениях скоро улягутся, а..."
1,Gemini,Первый день нового года прекрасно подходит для...
2,Taurus,Этот день как нельзя лучше подходит для улучше...
3,Cancer,"Первый день нового года, да еще понедельник бу..."
4,Leo,Львы не сорвутся в начале года с места в карье...
...,...,...
111150,Scorpio,Завтра состояние души Скорпиона будет напрямую...
111151,Sagittarius,Завтра есть смысл насильно заставить себя прео...
111152,Capricorn,Завтра Козерогу даже на романтическом свидании...
111153,Aquarius,Завтра звезды советуют Водолею не прятать голо...


In [87]:
Horoscope['short'] = Horoscope.text.apply(lambda name:  len(name) < 30 )
Horoscope.short.value_counts()

False    91470
True        71
Name: short, dtype: int64

In [88]:
Horoscope[Horoscope['short'] == True]

,sign,text,short
93875,Gemini,Начало новых приключений.,True
94334,Virgo,Дев ждет сумасшедшая любовь!,True
94360,Virgo,Дев все любят.,True
94740,Aquarius,Это сука любовь!,True
94921,Capricorn,День в ритме ву-ху.,True
...,...,...,...
107428,Aquarius,«Ты лучше всех».,True
107632,Aquarius,"В жопу эту любовь, короче",True
107838,Scorpio,Просто нанесите удар!,True
107849,Pisces,Да-да. Осталось немного!,True


In [89]:
Horoscope = Horoscope.loc[Horoscope['short'] == False].drop("short", axis = 1)
Horoscope

,sign,text
0,Aries,"Любые разногласия во мнениях скоро улягутся, а..."
1,Gemini,Первый день нового года прекрасно подходит для...
2,Taurus,Этот день как нельзя лучше подходит для улучше...
3,Cancer,"Первый день нового года, да еще понедельник бу..."
4,Leo,Львы не сорвутся в начале года с места в карье...
...,...,...
111150,Scorpio,Завтра состояние души Скорпиона будет напрямую...
111151,Sagittarius,Завтра есть смысл насильно заставить себя прео...
111152,Capricorn,Завтра Козерогу даже на романтическом свидании...
111153,Aquarius,Завтра звезды советуют Водолею не прятать голо...


## Посмотрим на самые популярные в гороскопах слова

In [ ]:
words = ""
stopwords = set(STOPWORDS)
for review in Horoscope.text.values:
    text = str(review)
    text = text.split()
    words += " ".join([(i.lower() + " ") for i in text])
    
cloud = WordCloud(width = 500, height = 500, background_color = 'white', stopwords = stopwords, min_font_size = 10)
cloud.generate(words)

plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(cloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.title("Horoscopes Word Cloud", fontsize = 16)
    
plt.show()

### В облаке мы видим упоминания дней недели, они нам не нужны, будем от них избавляться, так же нам не нужны упоминания знаков зодиака, их нам тоже стоит удалить, с них и начнем

Далее нам нужно найти все упоминания знаков зодиака

In [90]:
zodiac_signs = ['Рыбы', 'Водолей', 'Козерог', 'Стрелец', 'Скорпион', 'Весы',
                'Дева', 'Лев', 'Рак', 'Близнецы', 'Телец', 'Овен']

Все выписанные выше знаки зодиака находятся в нормальной форме, в тексте гороскопов они могут встречаться и в других формах, для того, чтобы правильно идентифицировать в тексте их все, нам нужно провести морфологический анализ.

Для этого идеально подойдет библиотека pymorphy2, при вызове метода MorphAnalyzer.parse() в разборе слова мы видим "тэг"(tag) ' - это набор граммем, характеризующих данное слово.

In [91]:
morph = pymorphy2.MorphAnalyzer(lang='ru')

In [92]:
test = morph.parse("гороскоп") 
test

[Parse(word='гороскоп', tag=OpencorporaTag('NOUN,inan,masc sing,nomn'), normal_form='гороскоп', score=0.666666, methods_stack=((DictionaryAnalyzer(), 'гороскоп', 34, 0),)),
 Parse(word='гороскоп', tag=OpencorporaTag('NOUN,inan,masc sing,accs'), normal_form='гороскоп', score=0.333333, methods_stack=((DictionaryAnalyzer(), 'гороскоп', 34, 3),))]

В данном случае теги "NOUN", "accs" и "nomn" означают, что это существительное в именительном или винительном падежах

###### С помощью атрибута Parse.lexeme можно получить все лексемы данного слова:

In [93]:
test[0].lexeme

[Parse(word='гороскоп', tag=OpencorporaTag('NOUN,inan,masc sing,nomn'), normal_form='гороскоп', score=1.0, methods_stack=((DictionaryAnalyzer(), 'гороскоп', 34, 0),)),
 Parse(word='гороскопа', tag=OpencorporaTag('NOUN,inan,masc sing,gent'), normal_form='гороскоп', score=1.0, methods_stack=((DictionaryAnalyzer(), 'гороскопа', 34, 1),)),
 Parse(word='гороскопу', tag=OpencorporaTag('NOUN,inan,masc sing,datv'), normal_form='гороскоп', score=1.0, methods_stack=((DictionaryAnalyzer(), 'гороскопу', 34, 2),)),
 Parse(word='гороскоп', tag=OpencorporaTag('NOUN,inan,masc sing,accs'), normal_form='гороскоп', score=1.0, methods_stack=((DictionaryAnalyzer(), 'гороскоп', 34, 3),)),
 Parse(word='гороскопом', tag=OpencorporaTag('NOUN,inan,masc sing,ablt'), normal_form='гороскоп', score=1.0, methods_stack=((DictionaryAnalyzer(), 'гороскопом', 34, 4),)),
 Parse(word='гороскопе', tag=OpencorporaTag('NOUN,inan,masc sing,loct'), normal_form='гороскоп', score=1.0, methods_stack=((DictionaryAnalyzer(), 'горос

Получим лексемы для всех знаком зодиака в одном списке:

In [94]:
all_zodiac_signs = []    
for sign in zodiac_signs:    
    horo = morph.parse(sign)[0]
    for word in horo.lexeme:
        all_zodiac_signs.append(word.word.capitalize())

In [95]:
len(all_zodiac_signs)

140

Затем стоит отсортировать список знаков зодиака, так как если, например, "дева" будет раньше, чем "девами" будет 2 срабатывания при замене слова в следующем модуле. Также, в разных падежах форма слова может оставаться неизменной, убираем дубли с помощью set

In [96]:
all_zodiac_signs=sorted(set(all_zodiac_signs), reverse=True)
len(all_zodiac_signs)

115

Заменим все знаки зодиака на "blank", так как есть вероятность того, что недобросовестные звёзды меняют только знак зодиака в своих прогнозах

In [97]:
Horoscope['Horoscope_step_1'] = Horoscope['text']
Horoscope.index = np.arange(len(Horoscope))
for horo_index in range(len(Horoscope)):
    for word in all_zodiac_signs:
        if word in Horoscope['Horoscope_step_1'][horo_index]:
            Horoscope['Horoscope_step_1'][horo_index] = Horoscope['Horoscope_step_1'][horo_index].replace(word, 'blank')

Проверка

In [98]:
Horoscope.Horoscope_step_1.apply(lambda name: sum([x in all_zodiac_signs for x in name.split()])).value_counts()

0    91470
Name: Horoscope_step_1, dtype: int64

###### Теперь необходимо проделать тоже самое с днями недели

In [99]:
week_days = ['понедельник', 'вторник', 'среда', 'четверг', 'пятница', 'суббота', 'воскресенье']

In [100]:
all_week_days = []
for day in week_days:
    unique_day = morph.parse(day)[0]
    for word in unique_day.lexeme:
        all_week_days.append(word.word)

In [101]:
len(all_week_days)

99

In [102]:
all_week_days=sorted(set(all_week_days), reverse=True)
len(all_week_days)

77

In [103]:
Horoscope.index = np.arange(len(Horoscope))
for horo_index in range(len(Horoscope)):
    for word in week_days:
        if word in Horoscope['Horoscope_step_1'][horo_index]:
            Horoscope['Horoscope_step_1'][horo_index] = \
            Horoscope['Horoscope_step_1'][horo_index].replace(word, 'week_day')

Проверка

In [104]:
Horoscope.Horoscope_step_1.apply(lambda name: sum([x in week_days for x in name.split()])).value_counts()

0    91470
Name: Horoscope_step_1, dtype: int64

Удаляем гороскопы, которые у нас стали одинаковыми

In [105]:
Horoscope.shape

(91470, 3)

In [106]:
Horoscope = Horoscope.drop_duplicates(subset=['Horoscope_step_1'])

In [107]:
Horoscope.shape

(77440, 3)

##### Теперь нужно убрать из данных смайлики

In [108]:
smiles = [
    '\u2764', '\U0001f618', '\u263a', '\U0001f60e', '\U0001f607', '\U0001f608', '\U0001f60d', '\U0001f60f', '\U0001f603', '\U0001f643'
]

In [109]:
Horoscope.drop_duplicates(subset = 'Horoscope_step_1').shape

(77440, 3)

In [125]:
Horoscope

,sign,text,test,for_drop
0,Aries,"Любые разногласия во мнениях скоро улягутся, а...",Любые разногласия во мнениях скоро улягутся а...,0
1,Gemini,Первый день нового года прекрасно подходит для...,Первый день нового года прекрасно подходит для...,0
2,Taurus,Этот день как нельзя лучше подходит для улучше...,Этот день как нельзя лучше подходит для улучше...,0
3,Cancer,"Первый день нового года, да еще понедельник бу...",Первый день нового года да еще понедельник бу...,0
4,Leo,Львы не сорвутся в начале года с места в карье...,Львы не сорвутся в начале года с места в карье...,1
...,...,...,...,...
91451,Leo,"Завтра Лев может испытывать желание поспорить,...",Завтра Лев может испытывать желание поспорить ...,1
91458,Pisces,Завтра Рыбы будут обеспокоены давлением со сто...,Завтра Рыбы будут обеспокоены давлением со сто...,1
91466,Sagittarius,Завтра есть смысл насильно заставить себя прео...,Завтра есть смысл насильно заставить себя прео...,0
91468,Aquarius,Завтра звезды советуют Водолею не прятать голо...,Завтра звезды советуют Водолею не прятать голо...,1


#### Дальше есть 2 пути:
* Можно разделить данные на 12 частей и обучить 12 разных моделей
* Убрать из данных упоминания знаков зодиака и сделать 1 модель

Я пробовал оба варианта, проблема 1го в количестве занимаемой памяти и недостаточности данных, модель получается недообученая.
Второй вариант значительно проще, требует меньше времени на обучение и места на диске, и, что самое главное, он генерирует гороскопы в разы лучше, я был вынужден выбрать его

### Вариант 1

Теперь нужно разделить датафрейм по знаку зодиака 

In [ ]:
for sign in Horoscope.sign.unique():
    df = Horoscope[Horoscope['sign'] == sign]
#     Horoscope.to_csv(f'D:/Horoscope_for_{sign}.csv', sep=',')
    print(len(df))
    df.reset_index(inplace=True, drop=True)
    f = open(f'{path}/Horoscope_for_{sign}.txt', 'w', encoding="utf8")
    for index in range(len(df)):
        f.write('Гороскоп: ' + df.text[index] + '\n')

### Вариант 2

In [147]:
Horoscope

,sign,text,test,for_drop
0,Aries,"Любые разногласия во мнениях скоро улягутся, а...",Любые разногласия во мнениях скоро улягутся а...,0
1,Gemini,Первый день нового года прекрасно подходит для...,Первый день нового года прекрасно подходит для...,0
2,Taurus,Этот день как нельзя лучше подходит для улучше...,Этот день как нельзя лучше подходит для улучше...,0
3,Cancer,"Первый день нового года, да еще понедельник бу...",Первый день нового года да еще понедельник бу...,0
4,Leo,Львы не сорвутся в начале года с места в карье...,Львы не сорвутся в начале года с места в карье...,1
...,...,...,...,...
91451,Leo,"Завтра Лев может испытывать желание поспорить,...",Завтра Лев может испытывать желание поспорить ...,1
91458,Pisces,Завтра Рыбы будут обеспокоены давлением со сто...,Завтра Рыбы будут обеспокоены давлением со сто...,1
91466,Sagittarius,Завтра есть смысл насильно заставить себя прео...,Завтра есть смысл насильно заставить себя прео...,0
91468,Aquarius,Завтра звезды советуют Водолею не прятать голо...,Завтра звезды советуют Водолею не прятать голо...,1


In [148]:
Horoscope['test'] = Horoscope.text.apply(lambda name: re.sub(r'[!@#$\n-.,]',' ', name))

<ipython-input-148-029454389a68>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Horoscope['test'] = Horoscope.text.apply(lambda name: re.sub(r'[!@#$\n-.,]',' ', name))


In [149]:
Horoscope['for_drop'] = Horoscope.test.apply(lambda name: sum([x in all_zodiac_signs for x in name.split()]))

<ipython-input-149-bc03cc5480d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Horoscope['for_drop'] = Horoscope.test.apply(lambda name: sum([x in all_zodiac_signs for x in name.split()]))


In [150]:
Horoscope.for_drop.value_counts()

1    35066
0    33569
2     5334
3     2971
4      466
5       31
6        2
9        1
Name: for_drop, dtype: int64

In [151]:
result = Horoscope.loc[Horoscope['for_drop'] == 0].drop(["test", "for_drop", "sign"], axis = 1)

In [157]:
result.reset_index(inplace=True, drop=True)

In [158]:
result

,text
0,"Любые разногласия во мнениях скоро улягутся, а..."
1,Первый день нового года прекрасно подходит для...
2,Этот день как нельзя лучше подходит для улучше...
3,"Первый день нового года, да еще понедельник бу..."
4,Сегодняшнее расположение звезд может усилить д...
...,...
33564,Настройтесь завтра провести приятный романтиче...
33565,"Если вам показалось, что ваши отношения с люби..."
33566,Уделите себе особое внимание. Измените прическ...
33567,Теплые взгляды и нежные речи... Что еще требуе...


In [160]:
f = open(f'{path}/result_data.txt', 'w', encoding="utf8")
for index in range(len(result)):
    f.write('Гороскоп: ' + result.text[index] + '\n')

### Начинаем обучение модели

In [ ]:
!pip install folium==0.2.1;
!pip install torch==1.4.0 > /dev/null
!pip install datasets transformers[sentencepiece]==3.5.0 > /dev/null

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
train = open('{path}/result_data.txt', 'r', encoding="utf8").readlines()

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts
!CUDA_VISIBLE_DEVICES=0

In [ ]:
!git clone https://github.com/sberbank-ai/ru-gpts.git ${HOME}/ru-gpts

In [ ]:
!touch ./results.txt

In [ ]:
!python ${HOME}/ru-gpts/pretrain_transformers.py \
--output_dir=/content/res \
--model_type=gpt2 \
--model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
--do_train \
--train_data_file=/Horoscope_for_dolbaeb.txt \
--do_eval \
--eval_data_file=/content/results.txt \
--per_gpu_train_batch_size 1 \
--gradient_accumulation_steps 1 \
--num_train_epochs 5 \
--block_size 1024 \
--overwrite_output_dir

### Смотрим на результат

In [ ]:
tok = GPT2Tokenizer.from_pretrained("/content/res")
model = GPT2LMHeadModel.from_pretrained("/content/res")
model.cuda();

In [ ]:
text = 'Гороскоп: '
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(),min_length=15, max_length=150, do_sample=True, top_k=6,
                     top_p=0.90, temperature=0.7, num_beams=None)

In [ ]:
tok.decode(out[0])

### Отлично, теперь полученую модель можно использовать